### Edit to recursively compute precision

Recursively compute precision for each class using the Eval column

In [1]:
import os
import csv
import pandas as pd
from pathlib import Path
import glob

### Precision computing TP/ALL 32

In [2]:
# Initialize directory - CHANGE IT ACCORDINGLY
recs_directory = "/mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/it_6/uncertainty/"

In [3]:
# List to store results
precision_data = []

for class_folder in os.listdir(recs_directory):
    class_path = os.path.join(recs_directory, class_folder)
    

    # Try to find the validation file:
    # 1) Look in the top_scoring subfolder
    top_scoring_path = os.path.join(class_path, "top_scoring")
    if os.path.isdir(top_scoring_path):
        txt_files = glob.glob(os.path.join(top_scoring_path, "*_Validation.txt"))
    else:
        # 2) If no top_scoring folder exists, look in directly in the class folder
        txt_files = glob.glob(os.path.join(class_path, "*_Validation.txt"))
        
    if not txt_files:
        continue # Skip if there is no Validation file
    
    txt_file = txt_files[0]  # Pick the first found file
    class_name = class_folder  # Use folder name as class name

    try:
        # Load the .txt file
        df = pd.read_csv(txt_file, delimiter='\t')

        # Compute precision: (count of 1s) / (total count)
        total_samples = len(df) # Not sure if use this or use only the ones I am scoring with 1 or 0 
        positive_samples = df["Eval"].astype(str).str.count("1").sum()
        precision = positive_samples / total_samples if total_samples > 0 else 0

        # Store the result
        precision_data.append({"class": class_name, "precision": round(precision, 4)})
    except Exception as e:
        print(f"Error processing {txt_file}: {e}")
        continue

# Convert list to DataFrame
precision_df = pd.DataFrame(precision_data)
# Save to CSV
precision_df.to_csv(os.path.join(recs_directory, "precision_it6_all.csv"), index=False)

print(f"Precision for all classes computed and saved")


Precision for all classes computed and saved


### Precision Ignoring the NaN - TP/TP+FP

In [4]:
import os
import glob
import pandas as pd

In [5]:
# Initialize directory
recs_directory = "/mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/it_6/uncertainty/"

In [6]:
# Collect results
precision_data = []

for class_folder in os.listdir(recs_directory):
    class_path = os.path.join(recs_directory, class_folder)

    # Look inside 'top_scoring' subfolder if it exists
    top_scoring_path = os.path.join(class_path, "top_scoring")
    if os.path.isdir(top_scoring_path):
        txt_files = glob.glob(os.path.join(top_scoring_path, "*_Validation.txt"))
    else:
        txt_files = glob.glob(os.path.join(class_path, "*_Validation.txt"))

    if not txt_files:
        continue

    txt_file = txt_files[0]
    class_name = class_folder

    try:
        # Read and process the Eval column

        df = pd.read_csv(txt_file, delimiter='\t', skipinitialspace=True)
        
        # Ensure Eval is numeric, invalid values become NaN
        df['Eval'] = pd.to_numeric(df['Eval'], errors='coerce')

        tp = (df['Eval'] == 1).sum()
        fp = (df['Eval'] == 0).sum()
        total = tp + fp

        precision = tp / total if total > 0 else 0

        precision_data.append({
            "class": class_name,
            "precision": round(precision, 4),
            "denominator (TP+FP)": total,
            "TP": int(tp),
            "FP": int(fp)
        })

    except Exception as e:
        print(f"Error processing {txt_file}: {e}")
        continue

# Save results
precision_df = pd.DataFrame(precision_data)
precision_df.to_csv(os.path.join(recs_directory, "precision_it6_ignore-nan.csv"), index=False)

print("Precision (ignoring NaN) computed and saved.")


Precision (ignoring NaN) computed and saved.
